# ASYNC MARGINS

In [ ]:
## THIS CELL SHOULD BE IN ALL NOTEBOOKS ##

import sys

import pandas as pd
from from_root import from_root
from ib_async import util

ROOT = from_root()
if str(ROOT) not in sys.path:
    sys.path.insert(1, str(ROOT))

pd.options.display.max_columns = None
pd.set_option("display.precision", 2)

In [ ]:
# SET ENVIRONMENTS AND IMPORTS
import asyncio
import math
from typing import Union

import nest_asyncio
import numpy as np
from ib_async import IB, MarketOrder, Option
from loguru import logger

from nse import RBI, NSEfnos, equity_iv_df
from utils import (
    arrange_df_columns,
    black_scholes,
    get_prec,
    load_config,
    make_contracts_orders,
    merge_and_overwrite_df,
)

nest_asyncio.apply()

config = load_config()

## Build a symbol

In [ ]:
# Fetch a symbol
symbol = "PNB"

In [ ]:
n = NSEfnos()
q = n.stock_quote_fno(symbol)

In [ ]:
dfe = equity_iv_df(q)

In [ ]:
# clean up zero IVs and dtes
mask = (dfe.iv > 0) & (dfe.dte > 0)
df = dfe[mask].reset_index(drop=True)

In [ ]:
df

## Append safe strikes

In [ ]:
def append_safe_strikes(df: pd.DataFrame) -> pd.DataFrame:
    """Appends safe-strikes and intrinsics from iv, undPrice and dte"""

    PUTSTDMULT = config.get("PUTSTDMULT")
    CALLSTDMULT = config.get("CALLSTDMULT")

    df_sp = pd.concat(
        [
            df,
            pd.Series(
                df.iv
                * df.undPrice
                * (df.dte / 365).apply(lambda x: math.sqrt(x) if x >= 0 else np.nan),
                name="sdev",
            ),
        ],
        axis=1,
    )

    # calculate safe strike with option price added
    safe_strike = np.where(
        df_sp.right == "P",
        (df_sp.undPrice - df_sp.sdev * PUTSTDMULT).astype("int"),
        (df_sp.undPrice + df_sp.sdev * CALLSTDMULT).astype("int"),
    )

    df_sp = df_sp.assign(safe_strike=safe_strike)

    # intrinsic value
    intrinsic = np.where(
        df_sp.right == "P",
        (df_sp.strike - df_sp.safe_strike).map(lambda x: max(0, x)),
        (df_sp.safe_strike - df_sp.strike).map(lambda x: max(0, x)),
    )

    df_sp = df_sp.assign(intrinsic=intrinsic)

    return df_sp

In [ ]:
df = append_safe_strikes(df)

## Append black-scholes price

In [ ]:
def append_black_scholes(df: pd.DataFrame) -> pd.DataFrame:
    """Appends black_scholed price to df"""

    rbi = RBI()
    risk_free_rate = rbi.repo_rate() / 100

    # Compute the black_scholes of option strike
    bsPrice = df.apply(
        lambda row: black_scholes(
            S=row["undPrice"],
            K=row["strike"],
            T=row["dte"] / 365,  # Convert days to years
            r=risk_free_rate,
            sigma=row["iv"],
            option_type=row["right"],
        ),
        axis=1,
    )

    df_out = df.assign(bsPrice=bsPrice)

    return df_out

In [ ]:
df = append_black_scholes(df)

In [ ]:
df.head()

## Append contract orders

In [ ]:
def append_cos(df: pd.DataFrame) -> pd.DataFrame:

    """Append contract and order fields"""

    dfo = make_contracts_orders(df)
    df = df.assign(contract=dfo.contract, order=dfo.order)

    return df

In [ ]:
df = append_cos(df)

In [ ]:
df

## Get margins

In [ ]:
async def get_one_margin(ib, contract, order, timeout):
    """Get margin with commissions within a time"""
    try:
        wif = await asyncio.wait_for(
            ib.whatIfOrderAsync(contract, order), timeout=timeout
        )
    except asyncio.TimeoutError:
        logger.error(f"{contract.localSymbol} wif timed out!")
        wif = None
    return wif


def margin_comm(r) -> dict:
    """Clean a result"""

    if r:
        margin = float(r.maintMarginChange)
        comm = min(float(r.commission), float(r.minCommission), float(r.maxCommission))
        if comm > 1e7:
            comm = np.nan
    else:
        margin = comm = np.nan

    return (margin, comm)


async def marginsAsync(
    df: pd.DataFrame, port: int, timeout: float = 2, eod: bool = True, ist: bool = True
) -> pd.DataFrame:
    """Gets async contracts from a df
    Args:
      df: dataframe with `contract` and `order` columns
      port: ib port
      timeout: time to wait. ~2 seconds for 10 rows
    Returns:
      a Dataframe with same index as input"""

    try:
        contracts = df.contract.to_list()
        orders = df.order.to_list()
    except ValueError as e:
        logging.error(f"df does not have contract or order.Error: {e}")
        return pd.DataFrame([])

    with IB().connect(port=port) as ib:

        # qualify contracts if there is no conId
        if df.contract.iloc[0].conId == 0:
            ib.qualifyContracts(*contracts)

        cos = zip(contracts, orders)

        tasks = [asyncio.create_task(get_one_margin(ib, c, o, timeout)) for c, o in cos]

        results = await asyncio.gather(*tasks)

    mcom = [margin_comm(r) for r in results]

    df1 = pd.DataFrame(mcom, columns=["margin", "comm"])
    df_mcom = df1.assign(contract=contracts)

    return df_mcom

In [ ]:
port = config.get("PORT")
df_mcom = await marginsAsync(df, port, timeout=2)

In [ ]:
df.head()

In [ ]:
df_mcom.head()

In [ ]:
df

## Get the expected price and rom

In [ ]:
def append_xPrice(df: pd.DataFrame) -> pd.DataFrame:

    """Append expected price, filter minimum rom and sort by likeliest"""

    # remove order column
    df = merge_and_overwrite_df(df, df_mcom).\
                drop(columns=['order'], errors='ignore')
    
    # get maxprice
    maxPrice = np.maximum(df.price, df.bsPrice)
    
    # get expected price
    xPrice = (df.intrinsic + maxPrice).apply(lambda x: max(get_prec(x, 0.05), 0.05))
    df = df.assign(xPrice = xPrice)
    
    # prevent divide by zero for rom
    margin = np.where(df.margin <= 0, np.nan, df.margin)
    
    # calculate rom
    rom = df.xPrice * df.lot / margin * 365 / df.dte
    df = df.assign(rom=rom)

    # ensure minimum expected ROM
    MINEXPROM = config.get('MINEXPROM')
    df = df[df.rom > MINEXPROM].reset_index(drop=True)

    # sort by likeliest
    df = df.loc[(df.xPrice / df.price).sort_values().index]

    return df

In [ ]:
df = append_xPrice(df)

In [ ]:
125-109